In [54]:
import os 
import pandas as pd
from bs4 import BeautifulSoup
import warnings

In [55]:
warnings.simplefilter(action='ignore', category=FutureWarning)

In [29]:
SCORE_DIR = 'data/scores'

In [30]:
box_scores = os.listdir(SCORE_DIR)

In [31]:
box_scores = [os.path.join(SCORE_DIR, f) for f in box_scores if f.endswith('.html')]

In [39]:
def parse_html(box_score):
    with open(box_score) as f:
        html = f.read()
    
    soup = BeautifulSoup(html)

    [s.decompose for s in soup.select('tr.over_header')]
    [s.decompose for s in soup.select('tr.thead')]

    return soup

In [40]:
def read_line_score(soup):
    line_score = pd.read_html(str(soup), attrs={'id':'line_score'})[0]

    cols = list(line_score.columns)
    cols[0] = 'team'
    cols[-1] = 'total'
    line_score.columns = cols

    line_score = line_score[['team', 'total']]

    return line_score

In [50]:
def read_stats(soup, team, stat):
    df = pd.read_html(str(soup), attrs={'id': f'box-{team}-game-{stat}'}, index_col=0)[0]
    df = df.apply(pd.to_numeric, errors='coerce')
    df.drop('Reserves', inplace=True)

    return df

In [243]:
def read_season_info(soup):
    nav = soup.select('#bottom_nav_container')[0]
    hrefs = [a['href'] for a in nav.find_all('a')]
    season = os.path.basename(hrefs[1]).split('_')[0]

    return season

In [251]:
base_cols = None
games = []

for box_score in box_scores:
    soup = parse_html(box_score)
    line_score = read_line_score(soup)
    teams = list(line_score['team'])

    summaries = []
    for team in teams:
        basic = read_stats(soup, team, 'basic')
        advanced = read_stats(soup, team, 'advanced')

        totals = pd.concat([basic.iloc[-1,:], advanced.iloc[-1,:]])
        totals = totals.reset_index().drop(['level_0'], axis=1).set_index('Starters')
        totals.index = totals.index.str.lower()

        maxes = pd.concat([basic.iloc[:-1,:].max(), advanced.iloc[:-1,:].max()])
        maxes = maxes.reset_index().drop(['level_0'], axis=1).set_index('Starters')
        maxes.index = maxes.index.str.lower() + '_max'

        summary = pd.concat([totals, maxes])
        summary['Team Totals'][37:] = summary[0][37:]
        summary.drop(0, axis=1, inplace=True)

        if base_cols is None:
            base_cols = list(summary.index.drop_duplicates(keep='first'))
            base_cols = [b for b in base_cols if 'bpm' not in b]

        summary.columns = [teams.index(team)]
        summary = summary.loc[base_cols]

        summaries.append(summary)

    summary = pd.concat(summaries, axis=1).T

    game = pd.concat([summary, line_score], axis=1)

    game['home'] = [0,1]
    game_opp = game.iloc[::-1].reset_index()

    game_opp.columns += '_opp'

    full_game = pd.concat([game, game_opp], axis=1)
    full_game['season'] = read_season_info(soup)

    full_game['date'] = os.path.basename(box_score)[:8]
    full_game['date'] = pd.to_datetime(full_game['date'], format='%Y%m%d')

    full_game['won'] = full_game['total'] > full_game['total_opp']

    games.append(full_game)

    if len(games) %  100 == 0:
        print(f'{len(games)} / {len(box_scores)}')

100 / 11520
200 / 11520
300 / 11520
400 / 11520
500 / 11520
600 / 11520
700 / 11520
800 / 11520
900 / 11520
1000 / 11520
1100 / 11520
1200 / 11520
1300 / 11520
1400 / 11520
1500 / 11520
1600 / 11520
1700 / 11520
1800 / 11520
1900 / 11520
2000 / 11520
2100 / 11520
2200 / 11520
2300 / 11520
2400 / 11520
2500 / 11520
2600 / 11520
2700 / 11520
2800 / 11520
2900 / 11520
3000 / 11520
3100 / 11520
3200 / 11520
3300 / 11520
3400 / 11520
3500 / 11520
3600 / 11520
3700 / 11520
3800 / 11520
3900 / 11520
4000 / 11520
4100 / 11520
4200 / 11520
4300 / 11520
4400 / 11520
4500 / 11520
4600 / 11520
4700 / 11520
4800 / 11520
4900 / 11520
5000 / 11520
5100 / 11520
5200 / 11520
5300 / 11520
5400 / 11520
5500 / 11520
5600 / 11520
5700 / 11520
5800 / 11520
5900 / 11520
6000 / 11520
6100 / 11520
6200 / 11520
6300 / 11520
6400 / 11520
6500 / 11520
6600 / 11520
6700 / 11520
6800 / 11520
6900 / 11520
7000 / 11520
7100 / 11520
7200 / 11520
7300 / 11520
7400 / 11520
7500 / 11520
7600 / 11520
7700 / 11520
7800 / 1

In [257]:
pd.set_option('display.max_columns', None)

In [267]:
games_df = pd.concat(games, ignore_index=True)

In [269]:
games_df.to_csv('nba_games.csv')